In [213]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import psycopg2
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from io import StringIO
import json
import psycopg2
from psycopg2.extras import Json
from psycopg2 import sql
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
import math, statistics

In [207]:
player = 'Derrick White'
opp = 'CHA'

In [208]:
# Connect to PostgreSQL and load the data
conn = psycopg2.connect(
    host = "localhost", 
    dbname = "mnrj", 
    user= "postgres", 
    password = "gwdb", 
    port = 5600
)
def load_nba(player):
    try:
        conn = psycopg2.connect(
            host="localhost",
            dbname="mnrj",
            user="postgres",
            password="gwdb",
            port=5600
        )
        # Use the connection to execute the query
        query = f"SELECT * FROM nba WHERE player = '{player}';"
        df = pd.read_sql(query, conn)
        conn.close()
        return df
    except psycopg2.Error as e:
        print(f"Error occurred while connecting to the database or executing query: {e}")
        return None   
def load_player_positions():
    try:
        conn = psycopg2.connect(
            host="localhost",
            dbname="mnrj",
            user="postgres",
            password="gwdb",
            port=5600
        )
        # Use the connection to execute the query
        query = f"SELECT * FROM latest_player_teams;"
        df = pd.read_sql(query, conn)
        conn.close()
        return df
    except psycopg2.Error as e:
        print(f"Error occurred while connecting to the database or executing query: {e}")
        return None
def load_game_stats(player):
    positions_df = load_player_positions()
    if positions_df is None:
        return pd.DataFrame()

    try:
        conn = psycopg2.connect(host="localhost", dbname="mnrj", user="postgres", password="gwdb", port=5600)
        query = f"""
        SELECT *
        FROM game_stats
        WHERE teammates_points::jsonb ? '{player}';  -- Checking if player key exists in JSON
        """
        df = pd.read_sql(query, conn)
        conn.close()

        # def aggregate_position_data(data, exclude_player):
        #     position_totals = {'G': 0, 'F': 0, 'C': 0}
        #     total_contribution = 0  # To hold the sum of all contributions for normalization
            
        #     # Sum values by position
        #     for player, value in data.items():
        #         pos = positions_df.loc[positions_df['player'] == player, 'pos'].values
        #         if pos:
        #             position_totals[pos[0]] += value
        #             total_contribution += value
            
        #     # Subtract the contribution of the excluded player after the total is calculated
        #     if exclude_player in data:
        #         player_pos = positions_df.loc[positions_df['player'] == exclude_player, 'pos'].values
        #         if player_pos:
        #             position_totals[player_pos[0]] -= data[exclude_player]
        #             total_contribution -= data[exclude_player]
            
        #     # Convert totals to percentages
        #     if total_contribution > 0:  # Avoid division by zero
        #         for position in position_totals:
        #             position_totals[position] = round(position_totals[position] / total_contribution, 2)  # Round to 2 decimal places

        #     return position_totals

        # Aggregate data based on positions, removing the current player
        # def aggregate_position_data(data, exclude_player):
        #     if exclude_player in data:
        #         del data[exclude_player]  # Remove the player from the data
            
        #     position_totals = {'G': 0, 'F': 0, 'C': 0}
        #     total_contribution = 0  # To hold the sum of all contributions for normalization

        #     # Sum values by position
        #     for player, value in data.items():
        #         pos = positions_df.loc[positions_df['player'] == player, 'pos'].values
        #         if pos:
        #             position_totals[pos[0]] += value
        #             total_contribution += value

        #     # Convert totals to percentages
        #     if total_contribution > 0:  # Avoid division by zero
        #         for position in position_totals:
        #             position_totals[position] = round(position_totals[position] / total_contribution, 2)  # Round to 2 decimal places

        #     return position_totals
            
        def aggregate_position_data(json_data, exclude_player):
            if exclude_player in json_data:
                del json_data[exclude_player]  # Remove the player from the data
            
            position_totals = {'G': 0, 'F': 0, 'C': 0}
            for player, value in json_data.items():
                pos = positions_df.loc[positions_df['player'] == player, 'pos'].values
                if pos.size > 0:
                    position_totals[pos[0]] += value

            return position_totals
        stats_fields = ['teammates_points', 'teammates_rebounds', 'teammates_assists', 'opponents_points', 'opponents_rebounds', 'opponents_assists',
                        'teammates_pr','teammates_pa','teammates_ar','opponents_pr','opponents_pa','opponents_ar','teammates_pra','opponents_pra']
        # Applying the transformation for each stats field
        for stat_field in stats_fields:
            df[stat_field] = df[stat_field].apply(lambda x: aggregate_position_data(x, player))
        
        for field in stats_fields:
            # Expand each dictionary into separate columns
            df_field = df[field].apply(pd.Series)
            df_field.columns = [f"{field}_{col}" for col in df_field.columns]  # Rename columns to include stat field
            df = pd.concat([df, df_field], axis=1)
            df.drop(field, axis=1, inplace=True)  # Drop the original column


        return df
    except psycopg2.Error as e:
        print(f"Error occurred while connecting to the database or executing query: {e}")

nba_data = load_nba(player)
game_stats = load_game_stats(player)
data = nba_data.merge(
    game_stats,
    on= ["team", "opp","date"]
)

/var/folders/c1/fw5mzqbs65d1bvrp1f8s6zc00000gn/T/ipykernel_69011/3886768634.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
/var/folders/c1/fw5mzqbs65d1bvrp1f8s6zc00000gn/T/ipykernel_69011/3886768634.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
/var/folders/c1/fw5mzqbs65d1bvrp1f8s6zc00000gn/T/ipykernel_69011/3886768634.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [209]:
# Define transformers for both scenarios
transformers_actual = [
    ('actual_scaler', StandardScaler(), ['mp', 'total_score','teammates_points_F', 'teammates_points_C', 'teammates_rebounds_G',
       'teammates_rebounds_F', 'teammates_rebounds_C', 'teammates_assists_G',
       'teammates_assists_F', 'teammates_assists_C', 'opponents_points_G',
       'opponents_points_F', 'opponents_points_C', 'opponents_rebounds_G',
       'opponents_rebounds_F', 'opponents_rebounds_C', 'opponents_assists_G',
       'opponents_assists_F', 'opponents_assists_C', 'teammates_pr_G',
       'teammates_pr_F', 'teammates_pr_C', 'teammates_pa_G', 'teammates_pa_F',
       'teammates_pa_C', 'teammates_ar_G', 'teammates_ar_F', 'teammates_ar_C',
       'opponents_pr_G', 'opponents_pr_F', 'opponents_pr_C', 'opponents_pa_G',
       'opponents_pa_F', 'opponents_pa_C', 'opponents_ar_G', 'opponents_ar_F',
       'opponents_ar_C', 'teammates_pra_G', 'teammates_pra_F',
       'teammates_pra_C', 'opponents_pra_G', 'opponents_pra_F',
       'opponents_pra_C']),  # Example features
    ('categorical', OneHotEncoder(handle_unknown='ignore'), ['opp'])
]

transformers_percentage = [
    ('percentage_scaler', MinMaxScaler(), ['teammates_points_F', 'teammates_points_C', 'teammates_rebounds_G',
       'teammates_rebounds_F', 'teammates_rebounds_C', 'teammates_assists_G',
       'teammates_assists_F', 'teammates_assists_C', 'opponents_points_G',
       'opponents_points_F', 'opponents_points_C', 'opponents_rebounds_G',
       'opponents_rebounds_F', 'opponents_rebounds_C', 'opponents_assists_G',
       'opponents_assists_F', 'opponents_assists_C', 'teammates_pr_G',
       'teammates_pr_F', 'teammates_pr_C', 'teammates_pa_G', 'teammates_pa_F',
       'teammates_pa_C', 'teammates_ar_G', 'teammates_ar_F', 'teammates_ar_C',
       'opponents_pr_G', 'opponents_pr_F', 'opponents_pr_C', 'opponents_pa_G',
       'opponents_pa_F', 'opponents_pa_C', 'opponents_ar_G', 'opponents_ar_F',
       'opponents_ar_C', 'teammates_pra_G', 'teammates_pra_F',
       'teammates_pra_C', 'opponents_pra_G', 'opponents_pra_F',
       'opponents_pra_C']),  # Example features
    ('actual_scalar', StandardScaler(), ['mp','total_score']),
    ('categorical', OneHotEncoder(handle_unknown='ignore'), ['opp'])
]

In [220]:
df = data

# Specify features and target
features = ['opp', 'mp', 'total_score','teammates_points_F', 'teammates_points_C', 'teammates_rebounds_G',
       'teammates_rebounds_F', 'teammates_rebounds_C', 'teammates_assists_G',
       'teammates_assists_F', 'teammates_assists_C', 'opponents_points_G',
       'opponents_points_F', 'opponents_points_C', 'opponents_rebounds_G',
       'opponents_rebounds_F', 'opponents_rebounds_C', 'opponents_assists_G',
       'opponents_assists_F', 'opponents_assists_C', 'teammates_pr_G',
       'teammates_pr_F', 'teammates_pr_C', 'teammates_pa_G', 'teammates_pa_F',
       'teammates_pa_C', 'teammates_ar_G', 'teammates_ar_F', 'teammates_ar_C',
       'opponents_pr_G', 'opponents_pr_F', 'opponents_pr_C', 'opponents_pa_G',
       'opponents_pa_F', 'opponents_pa_C', 'opponents_ar_G', 'opponents_ar_F',
       'opponents_ar_C', 'teammates_pra_G', 'teammates_pra_F',
       'teammates_pra_C', 'opponents_pra_G', 'opponents_pra_F',
       'opponents_pra_C']
target = 'pts'

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=20)

preprocessor_actual = ColumnTransformer(transformers=transformers_actual)
preprocessor_percentage = ColumnTransformer(transformers=transformers_percentage)

# Create Pipelines
pipeline_actual = Pipeline([
    ('preprocessor', preprocessor_actual),
    ('regressor', RandomForestRegressor())
])

pipeline_percentage = Pipeline([
    ('preprocessor', preprocessor_percentage),
    ('regressor', RandomForestRegressor())])

# Fit the model
pipeline_actual.fit(X_train, y_train)
pipeline_percentage.fit(X_train, y_train)

# Predict and evaluate the model
y_pred_a = pipeline_actual.predict(X_test)
y_pred_p = pipeline_percentage.predict(X_test)
mse_a = mean_squared_error(y_test, y_pred_a)
mse_p = mean_squared_error(y_test, y_pred_p)
print(f'Root Mean Squared Error: {math.sqrt(mse_a)}')
print(f'Root Mean Squared Error: {math.sqrt(mse_p)}')
print(y_pred_a)
print(statistics.mean(y_pred_p))

Root Mean Squared Error: 6.417271616504946
Root Mean Squared Error: 6.665055013526395
[16.81 17.02 19.19 15.95 14.22 12.04 14.69 17.52 16.38 13.87 16.83 15.68
 14.45 14.07 14.88 15.58 17.43 16.84 18.5  13.12 21.36 19.22 15.91 17.36]
16.301666666666666
